In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Dense
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.utils import np_utils, print_summary
import pandas as pd
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import keras.backend as K

D:\ProgramData\Miniconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = pd.read_csv("data.csv")
data

,pixel_0000,pixel_0001,pixel_0002,pixel_0003,pixel_0004,pixel_0005,pixel_0006,pixel_0007,pixel_0008,pixel_0009,...,pixel_1015,pixel_1016,pixel_1017,pixel_1018,pixel_1019,pixel_1020,pixel_1021,pixel_1022,pixel_1023,character
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
dataset = np.array(data)
dataset

array([[ 0,  0,  0, ...,  0,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  1],
       ...,
       [ 0,  0,  0, ...,  0,  0, 36],
       [ 0,  0,  0, ...,  0,  0, 36],
       [ 0,  0,  0, ...,  0,  0, 36]], dtype=int64)

In [4]:
np.random.shuffle(dataset)

x = dataset
y = dataset

x = x[:, 0:1024]
y = y[:, 1024]

x_train = x[0:70000, :]
x_train = x_train/255.0

x_test = x[70000:72001, :]
x_test = x_test/255.0

#reshape
y = y.reshape(y.shape[0],1)

y_train = y[0:70000, :]
y_train = y_train.T

y_test = y[70000:72001, :]
y_test = y_test.T

In [5]:
print("Number of training examples: "+ str(x_train.shape[0]))
print("Number of testing examples: "+ str(x_test.shape[0]))
print("x train shape: "+ str(x_train.shape))
print("x test shape: "+ str(x_test.shape))
print("y train shape: "+ str(y_train.shape))
print("y test shape: "+ str(y_train.shape))


Number of training examples: 70000
Number of testing examples: 2000
x train shape: (70000, 1024)
x test shape: (2000, 1024)
y train shape: (1, 70000)
y test shape: (1, 70000)


In [6]:
image_x = 32
image_y = 32

train_y = np_utils.to_categorical(y_train)
test_y = np_utils.to_categorical(y_test)

train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])

x_train = x_train.reshape(x_train.shape[0], image_x, image_y, 1)
x_test = x_test.reshape(x_test.shape[0], image_x, image_y, 1)


In [7]:
print("X train shape: " + str(x_train.shape))
print("Y train shape: " + str(train_y.shape))

X train shape: (70000, 32, 32, 1)
Y train shape: (70000, 37)


In [8]:
def keras_model(image_x, image_y):
    num_of_classes = 37
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (5,5), input_shape = (image_x, image_y, 1), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2,2), strides= (2,2), padding = "same"))
    model.add(Conv2D(filters = 64, kernel_size = (5,5), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (5,5), strides= (5,5), padding = "same"))
    
    model.add(Flatten())
    
    model.add(Dense(num_of_classes, activation = "softmax"))
    model.compile( loss = "categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
    
    filepath = "devnagri_model.h5"
    
    checkpoint1 = ModelCheckpoint(filepath, monitor = 'val_acc', verbose= 1, save_best_only = True, mode= "max")
    callbacks_list = [checkpoint1]
    
    return model, callbacks_list


In [9]:
model, callbacks_list = keras_model(image_x, image_y)

model.fit(x_train, train_y, validation_data = (x_test, test_y), epochs=5, batch_size=64, callbacks = callbacks_list)
scores = model.evaluate(x_test, test_y, verbose=0)

print("CNN Error: %.2f%%" % (100-scores[1]*100))

print_summary(model)

model.save("devnagri_model.h5")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 70000 samples, validate on 2000 samples
Epoch 1/5
70000/70000 [==============================] - 69s 988us/step - loss: 0.7873 - acc: 0.7781 - val_loss: 0.3041 - val_acc: 0.9035

Epoch 00001: val_acc improved from -inf to 0.90350, saving model to devnagri_model.h5
Epoch 2/5
70000/70000 [==============================] - 71s 1ms/step - loss: 0.2359 - acc: 0.9285 - val_loss: 0.2073 - val_acc: 0.9365

Epoch 00002: val_acc improved from 0.90350 to 0.93650, saving model to devnagri_model.h5
Epoch 3/5
70000/70000 [==============================] - 64s 919us/step - loss: 0.1564 - acc: 0.9522 - val_loss: 0.1768 - val_acc: 0.9510

Epoch 00003: val_acc improved from 0.93650 to 0.95100, saving model to devnagri_model.h5
Epoch 4/5
70000/70000 [==============================] - 69s 987us/step - loss: 0.1160 - acc: 0.9646 - val_loss: 0.1355 - val_acc: 0.9590

Epoch 00004: 

In [ ]:
# application
